# Defect Trend Analysis

This notebook visualizes **before vs. after** trends for production bugs and customer complaints, supporting the Nu Skin Cosmetics Security Enhancement case study.

**Goals**
- Load monthly metrics (bugs, complaints)
- Visualize the trend before and after QA process improvements
- Quantify percentage changes and summarize impact

> If `../results/defect_summary.csv` is not present, the notebook will create a realistic synthetic dataset.

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
plt.rcParams['figure.figsize'] = (10, 4)


In [ ]:
# Load metrics if available, otherwise synthesize realistic data
results_path = Path('../results/defect_summary.csv')
if results_path.exists():
    df = pd.read_csv(results_path, parse_dates=['month'])
else:
    months = pd.date_range('2023-01-01', periods=18, freq='MS')
    phase = ['before']*12 + ['after']*6
    bugs_before = [120,118,121,119,122,117,123,120,118,119,121,120]
    complaints_before = [50,49,52,50,51,48,50,49,50,51,50,50]
    import numpy as np
    bugs_after = (np.mean(bugs_before) * 0.60 + np.random.normal(0, 3, 6)).round().astype(int)
    complaints_after = (np.mean(complaints_before) * 0.80 + np.random.normal(0, 2, 6)).round().astype(int)
    bugs = list(bugs_before) + list(bugs_after)
    complaints = list(complaints_before) + list(complaints_after)
    df = pd.DataFrame({'month': months, 'phase': phase, 'production_bugs': bugs, 'customer_complaints': complaints})

df.head()


In [ ]:
# Compute deltas and percent improvements
grp = df.groupby('phase').mean(numeric_only=True)
bug_reduction = (1 - grp.loc['after','production_bugs'] / grp.loc['before','production_bugs']) * 100
complaint_reduction = (1 - grp.loc['after','customer_complaints'] / grp.loc['before','customer_complaints']) * 100
summary = pd.DataFrame({
    'metric': ['production_bugs','customer_complaints'],
    'before_mean': [grp.loc['before','production_bugs'], grp.loc['before','customer_complaints']],
    'after_mean': [grp.loc['after','production_bugs'], grp.loc['after','customer_complaints']],
    'percent_change': [bug_reduction, complaint_reduction]
})
summary.round(2)


In [ ]:
# Plot: Production bugs trend
fig1, ax1 = plt.subplots()
ax1.plot(df['month'], df['production_bugs'], marker='o')
ax1.axvline(df.loc[df['phase']=='after','month'].min(), color='gray', linestyle='--', label='Improvement Start')
ax1.set_title('Production Bugs per Month')
ax1.set_xlabel('Month')
ax1.set_ylabel('Bugs')
ax1.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Plot: Customer complaints trend
fig2, ax2 = plt.subplots()
ax2.plot(df['month'], df['customer_complaints'], marker='o')
ax2.axvline(df.loc[df['phase']=='after','month'].min(), color='gray', linestyle='--', label='Improvement Start')
ax2.set_title('Customer Complaints per Month')
ax2.set_xlabel('Month')
ax2.set_ylabel('Complaints')
ax2.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Optional: save outputs to ../results for README
out_dir = Path('../results')
out_dir.mkdir(parents=True, exist_ok=True)
df.to_csv(out_dir/'defect_summary.csv', index=False)
fig1.savefig(out_dir/'production_bugs_trend.png', dpi=200, bbox_inches='tight')
fig2.savefig(out_dir/'customer_complaints_trend.png', dpi=200, bbox_inches='tight')
print('Saved: defect_summary.csv, production_bugs_trend.png, customer_complaints_trend.png')


---
**Notes**
- This notebook is intentionally lightweight and uses only `matplotlib` for portability.
- Update the `results/defect_summary.csv` file with real metrics to reflect your environment.
- The dashed vertical line marks the start of the improvement period.